In [2]:
# System imports
import os
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import hvplot.pandas
import panel as pn
from pathlib import Path
from dotenv import load_dotenv
from panel.interact import interact
from panel import widgets
from string import digits
import csv
import json
import numpy as np
import dateparser
import panel as pn
from panel.interact import interact, interactive, fixed, interact_manual

pn.extension()

%matplotlib inline

In [3]:
# Local imports
import sys
sys.path.append("../lib2")

from Constants import Constants
from PreprocessingTools import MappingFunctions
from ProcessingTools import DateProcessingTools


In [4]:
# Construct the tools
debug_level = 0
constants = Constants()
tool_data_processing = DateProcessingTools(debug_level)
tool_lookup_tables = MappingFunctions(debug_level)


In [5]:
# Import all preprocessed data
atlas           = pd.read_pickle(constants.PREPROCESSED_ATLAS_FILE_PATH)
forecast        = pd.read_pickle(constants.PREPROCESSED_FORECAST_DATA_FILE_PATH)
revenue2020     = pd.read_pickle(constants.PREPROCESSED_REVENUE2020_FILE_PATH)
revenue2020A    = pd.read_pickle(constants.PREPROCESSED_REVENUE2020A_FILE_PATH)
atlas2          = pd.read_pickle(constants.PREPROCESSED_ATLAS_2_FILE_PATH)


In [6]:
load_dotenv()
mapbox_token = os.getenv("MAPBOX_API_KEY")

In [7]:
atlas2.head()
avg_price_df = atlas2.reset_index().groupby(by=['Subscription', 'Customers']).mean()
avg_price_df

Number of Users  Invoice Amount        Lat  \
Subscription Customers                                                    
1 Year       University 1                NaN    48500.000000  35.194648   
             University 10              95.0     1250.001667  37.423158   
             University 101             50.0     1250.000000  36.300110   
             University 104             25.0     2750.000000  45.507856   
             University 105            310.0     2775.000000  27.926018   
...                                      ...             ...        ...   
6 Months     University 420             10.0      800.000000  38.380263   
             University 423            500.0     3250.000000  28.062000   
             University 427            150.0     1500.000000  38.978405   
             University 437            500.0     3250.000000        NaN   
             University 443             50.0      270.000000        NaN   

                                   Long  
Subscription Customers                   
1 Year       University 1    -85.785180  
             University 10   -95.088468  
             University 101  -82.294030  
             University 104 -122.690794  
             University 105  -82.487126  
...                                 ...  
6 Months     University 420  -81.767141  
             University 423  -82.413225  
             University 427  -79.949417  
             University 437         NaN  
             University 443         NaN  

[464 rows x 4 columns]

In [8]:
# Define Panel Visualization Functions
# Group by year and . and then create a new dataframe of the mean values

def by_subscription():
    "Group By Year and mean values."
    avg_price_df = atlas2.reset_index().groupby(by=['Subscription', 'Customers']).mean()
    sfo_grouped_month_and_02_Anonymized_plot = avg_price_df.hvplot.line(
    x='Customers',
    y='Invoice Amount',
    xlabel= 'Customers',
    ylabel='Invoice Amount',
    groupby='Subscription', 
    width=600
    ).opts(yformatter='%0f')
    return sfo_grouped_month_and_02_Anonymized_plot

def top_most_paying_customers():
    "Top 10 Most Paying Customers"
    customers = avg_price_df.groupby(["Invoice Amount", "Customers"]).max()
    top_10_highest_customer = customers.sort_values(by='Invoice Amount', ascending=False).head(10)
    top_10_highest_customer = top_10_highest_customer.reset_index()
    
    top_10_highest_customer_plot = top_10_highest_customer.hvplot.bar(
    x='Customers',
    y='Invoice Amount',
    rot=90,
    title='Top 10 Highest Customers')
    return top_10_highest_customer_plot

def customers_address():
    "Customers Pings"
    total_data = atlas2.groupby(["Customers"]).sum()
    total_data.reset_index(inplace=True)
    total_data.dropna(inplace=True)
    px.set_mapbox_access_token(mapbox_token)
    map_pings = px.scatter_mapbox(
    total_data,
    lat="Lat",
    lon="Long",
    size="Invoice Amount",
    color="Customers"
)
    return map_pings

In [9]:
# by_subscription()
# p_most_paying_customers()
customers_address()

In [10]:
map_plt = pn.Column(customers_address())

#yearly_market_analysis = pn.Column(housing_units_per_year, average_gross_rent, average_sales_price())
customer_analysis = pn.Column((top_most_paying_customers, by_subscription))
#parallel_plot_analysis = pn.Column(parallel_categories(), parallel_coordinates())

panel_content = pn.Tabs(
                        ('Welcome', map_plt),
                        ('Customer Analysis', customer_analysis)
)

panel_content


pn.extension('plotly')



Tabs
    [0] Column
        [0] Plotly(Figure)
    [1] Column
        [0] Str(tuple)

In [11]:
panel_content.servable()

Tabs
    [0] Column
        [0] Plotly(Figure)
    [1] Column
        [0] Str(tuple)

In [15]:
panel_content.servable()


Tabs
    [0] Column
        [0] Plotly(Figure)
    [1] Column
        [0] Str(tuple)